In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import MESS
import numpy as np
import pandas as pd

from IPython.display import display

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Use the Reunion spider data for testing
First quickly munge the spider data into the form we need.

In [2]:
sp_dir = "/home/isaac/SGD_empirical/empirical_reunion_spiders/"
sp_abunds = pd.read_csv(sp_dir + "spider_abunds_df.txt", index_col=0, header=0)
sp_pis = pd.read_csv(sp_dir + "spider.pis", sep=" ", index_col=1, header=None)
sp_pis.index = [x.split("/")[1].split(".")[0] for x in sp_pis.index]
sp_df = pd.concat([sp_abunds, sp_pis], axis=1, sort=False)
sp_df.columns = ["abundance", "pi"]
sp_df.to_csv(sp_dir + "spider.dat", header=True)
sp_df[:5]

,abundance,pi
GL-01a,150,0.011126
GL-01b,107,0.023247
GL-01c,32,0.000252
GL-02,2,0.000000
GL-03,200,0.028499


In [ ]:
simfile = "/home/isaac/Continuosity/MESS/analysis/full-parameter-estimation/SIMOUT.txt"

rgr = MESS.inference.Regressor(empirical_df=sp_df, simfile=simfile, algorithm="rf")
#rgr.set_targets(target_list="_lambda")
rgr.predict(select_features=True, quick=False, verbose=True)

Selecting features:
  alpha	['S', 'abund_h1', 'abund_h2', 'abund_h3', 'abund_h4', 'pi_h1', 'pi_h2', 'pi_h3', 'pi_h4', 'mean_pi', 'std_pi', 'skewness_pi', 'kurtosis_pi', 'median_pi', 'iqr_pi', 'abundance_pi_cor', 'SGD_1', 'SGD_2']
  J_m	['S', 'SGD_0']
  ecological_strength	[]
  m	['S', 'abund_h1', 'abund_h2', 'abund_h3', 'abund_h4', 'mean_pi', 'std_pi', 'SGD_0', 'SGD_5']
  speciation_prob	[]
  _lambda	['S', 'abund_h1', 'abund_h2', 'abund_h3', 'abund_h4', 'mean_pi', 'std_pi', 'iqr_pi', 'abundance_pi_cor']
All selected features: S abund_h1 abund_h2 abund_h3 abund_h4 pi_h1 pi_h2 pi_h3 pi_h4 mean_pi std_pi skewness_pi kurtosis_pi median_pi iqr_pi abundance_pi_cor SGD_0 SGD_1 SGD_2 SGD_5
Finding best model parameters.
Fitting 4 folds for each of 100 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 22.5min


In [26]:
rgr._cvsearch.



RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

# Developing some kind of prediction uncertainty

## Predicting error with another ML model
This is a clever idea, not sure how kosher it is: https://qucit.com/a-simple-technique-to-estimate-prediction-intervals-for-any-regression-model_en/. Here's the gist: 

    # split the data in train a validation set
    X1, X2, y1, y2 = train_test_split(X, y, test_size=0.5)
    # base_model can be any regression model, a  
    # sklearn.ensemble.GradientBoostingRegressor for instance
    base_model.fit(X1, y1)
    base_prediction = base_model.predict(X2)
    # compute the prediction error vector on the validation set
    validation_error = (base_prediction - y2) ** 2
    error_model.fit(X2, validation_error)

    # compute the mean and standard deviation of the distribution
    mean = base_model.predict(X_test)
    st_dev = error_model.predict(X_test)

## skgarden.RandomForestQuantileRegressor
https://github.com/scikit-garden/scikit-garden
https://scikit-garden.github.io/examples/QuantileRegressionForests/

In [ ]:
### Use QuantileForests for quantile estimation
from skgarden import RandomForestQuantileRegressor
rfqr = RandomForestQuantileRegressor(random_state=0)
rfqr.fit(X, y)
y_mean = rfqr.predict(X)
y_median = rfqr.predict(X, 50)

## sklearn GB has quantile regression built in (sort of)
This works okay for GB, I guess.

In [24]:
from sklearn.ensemble import GradientBoostingRegressor

alpha = 0.95

clf = GradientBoostingRegressor(loss='quantile', alpha=alpha,
                                n_estimators=1000, max_depth=5,
                                learning_rate=.1, min_samples_leaf=9,
                                min_samples_split=9)

clf.fit(rgr.X, rgr.y)

# Make the prediction on the meshed x-axis
y_upper = clf.predict(rgr.empirical_sumstats)

clf.set_params(alpha=1.0 - alpha)
clf.fit(rgr.X, rgr.y)

# Make the prediction on the meshed x-axis
y_lower = clf.predict(rgr.empirical_sumstats)

clf.set_params(loss='ls')
clf.fit(rgr.X, rgr.y)

# Make the prediction on the meshed x-axis
y_pred = clf.predict(rgr.empirical_sumstats)

print(y_upper, y_lower, y_pred)

(array([0.99383061]), array([0.87963315]), array([1.03449941]))


## forestci: Again, not sure if this is useful


cd /home/isaac/Continuosity/MESS/util
git clone https://github.com/scikit-learn-contrib/forest-confidence-interval.git
cd forest-confidence-interval
python setup.py install

In [14]:
import forestci as fci
import sklearn.model_selection
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(rgr.X, rgr.y)
v = fci.random_forest_error(rgr.best_model, X_train, X_test)
v

array([0.01132867, 0.00425331, 0.01679821, ..., 0.00617017, 0.01379471,
       0.00411115])

In [20]:
v.shape
X_test.shape

(2994, 27)